In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [ ]:
df_initial = pd.read_csv('../../dataset/initial_balanced_tweets.csv')
df_pseudo = pd.read_csv('../../dataset/pseudo_labels_98pct_confidence.csv')

In [4]:
X = df_initial['tweet']
y = df_initial['sentiment']

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [5]:
print("Train",y_train.value_counts())
print("Val",y_val.value_counts())

'''
Train sentiment
0    12562
2    12561
1    12561
Name: count, dtype: int64
Val sentiment
2    3141
1    3141
0    3140
Name: count, dtype: int64
'''

Train sentiment
0    12562
2    12561
1    12561
Name: count, dtype: int64
Val sentiment
2    3141
1    3141
0    3140
Name: count, dtype: int64


'\nTrain sentiment\n0    12562\n2    12561\n1    12561\nName: count, dtype: int64\nVal sentiment\n2    3141\n1    3141\n0    3140\nName: count, dtype: int64\n'

In [6]:
gold_train = pd.concat([X_train,y_train],axis=1)
gold_val = pd.concat([X_val,y_val],axis=1)

In [7]:
df_combined=pd.concat([gold_train,df_pseudo],axis=0)

In [8]:
print("Val:",len(gold_val))
print("Train:",len(gold_train))
print("Pseudo:",len(df_pseudo))
print("Combined:",len(df_combined))

'''
Val: 9422
Train: 37684
Pseudo: 81737
Combined: 119421
'''

Val: 9422
Train: 37684
Pseudo: 81737
Combined: 119421


'\nVal: 9422\nTrain: 37684\nPseudo: 81737\nCombined: 119421\n'

In [9]:
X_combined=df_combined['tweet']
y_combined=df_combined['sentiment']
print(y_combined.value_counts())

'''
sentiment
2    44516
1    40943
0    33962
Name: count, dtype: int64
'''

sentiment
2    44516
1    40943
0    33962
Name: count, dtype: int64


'\nsentiment\n2    44516\n1    40943\n0    33962\nName: count, dtype: int64\n'

In [10]:
# Balance with Synthetic Tweets

synth_df = pd.read_parquet("hf://datasets/TimKoornstra/synthetic-financial-tweets-sentiment/data/train-00000-of-00001.parquet")

# Previously balanced original train/val set using:
# bearish = synth_df[synth_df['sentiment'] == 2].head(8826)
# neutral = synth_df[synth_df['sentiment'] == 0].head(5187)

#bearish_start=8826
neutral_start=5187

bullish_needed=(44516-40943)
neutral_needed=(44516-33962)

bullish = synth_df[synth_df['sentiment'] == 1].head(bullish_needed)
neutral = synth_df[synth_df['sentiment'] == 0].iloc[neutral_start:].head(neutral_needed)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [11]:
df_combined=pd.concat([df_combined,bullish,neutral],axis=0, ignore_index=True)

X_combined=df_combined['tweet']
y_combined=df_combined['sentiment']
print(y_combined.value_counts())

'''
sentiment
0    44516
2    44516
1    44516
Name: count, dtype: int64
'''

sentiment
0    44516
2    44516
1    44516
Name: count, dtype: int64


'\nsentiment\n0    44516\n2    44516\n1    44516\nName: count, dtype: int64\n'

In [ ]:
df_combined.to_csv("../../dataset/semisupervised_training.csv", index=False)
print("Saved Semisupervised Training Set => Original Training Split + Pseudolabels + New Synthetic Balancing Tweets")

gold_val.to_csv("../../dataset/semisupervised_validation.csv", index=False)
print("Saved Semisupervised Validation Set => just the Original Validation Split")

Saved Semisupervised Training Set => Original Training Split + Pseudolabels + New Synthetic Balancing Tweets
Saved Semisupervised Validation Set => just the Original Validation Split
